In [1]:
import pandas as pd
import json
import requests

## Набор данных

In [173]:
path_dataset = "D:/datasets/flickr-30k-images"
path_captions = "D:/YandexDisk/datasets/flickr-30k-images-metadata.csv"

path_prepared_captions = "D:/flickr-30k-images-metadata.csv"

In [47]:
df_captions = pd.read_csv(path_captions, delimiter='|')
df_captions.head(5)

,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their...
1,1000092795.jpg,1,"Two young , White males are outside near many..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard .
3,1000092795.jpg,3,A man in a blue shirt standing in a garden .
4,1000092795.jpg,4,Two friends enjoy time spent together .


In [98]:
df_captions.iat[1, 2]

'Two young, white males are outside near many bushes'

## Подготовка к переводу

– Первая буква предложения – заглавная.
– Удалить пробелы в начале и в конце.
– Удалить точку в конце.
– Удалить точку.
– Удалить пробел перед запятой.
– Удалить пробел перед апострофом.
– Удалить многоточие.
– Удалить вопросительный знак.
– Удалить восклицательный знак.

In [54]:
count_chars = 0

for idx in df_captions.index:
    df_captions.iat[idx, 2] = df_captions.iat[idx, 2]\
        .strip()\
        .capitalize()\
        .replace(" .", "")\
        .replace(".", "")\
        .replace(" ,", ",")\
        .replace(" '", "'")\
        .replace(" ...", "")\
        .replace("...", "")\
        .replace(" ?", "")\
        .replace("?", "")\
        .replace(" !", "")\
        .replace("!", "")

    count_chars += len(df_captions.iloc[idx, 2])

print(count_chars)

9871649


## Найти кавычки

In [107]:
idx_with_quotations = []

for idx in df_captions.index:
    find_idx = df_captions.iat[idx, 2].find('"')
    if find_idx != -1:
        idx_with_quotations.append(idx)

len(idx_with_quotations)

840

In [6]:
some_idx = idx_with_quotations[0]
df_captions.iat[some_idx, 2]

'The " white out " conditions of snow on the ground seem to almost obliverate the details of a man dressed for the cold weather in a heavy jacket and red hat riding a bicycle in a suburban neighborhood'

## Удалить изображения, у которых в описании есть кавычки

In [7]:
df_captions_with_quotations = df_captions.loc[idx_with_quotations, :]
images_to_delete = df_captions_with_quotations["image_name"].unique()
images_to_delete.shape

(672,)

In [8]:
df_captions.shape

(158915, 3)

In [9]:
df_captions = df_captions[~df_captions.image_name.isin(images_to_delete)]
df_captions.shape

(155555, 3)

In [11]:
df_captions.head(5)

,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their ...
1,1000092795.jpg,1,"Two young, white males are outside near many b..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard
3,1000092795.jpg,3,A man in a blue shirt standing in a garden
4,1000092795.jpg,4,Two friends enjoy time spent together


## Найти цифры

In [10]:
def is_there_number(string):
    for i in string.split():
        if i.isdigit():
            return True

    return False

In [15]:
string = "I am 45 years old"
print(is_there_number(string))

True


In [16]:
idx_with_numbers = []

for idx in df_captions.index:
    if is_there_number(df_captions.iat[idx, 2]):
        idx_with_numbers.append(idx)

len(idx_with_numbers)

1491

In [37]:
some_idx = idx_with_numbers[20]
df_captions.iat[some_idx, 2]

'A dog is wearing a red number 6 and running through a field of green grass while being chased by a bird'

## Сохранить dataframe в файл

In [177]:
df_captions.to_csv(path_prepared_captions, sep='|', encoding='utf-8', index=False)

## Перевод

In [3]:
IAM_TOKEN = "ВАШ ТОКЕН"
BEARER = "Bearer " + IAM_TOKEN
FOLDER_ID = "ВАШ КАТАЛОГ"

HEADERS = {
    "Content-type": "application/json",
    "Authorization": BEARER
}

def translate_with_yandex(text):
    data = {
        "folder_id": FOLDER_ID,
        "texts": text,
        "targetLanguageCode": "ru"
    }

    response = requests.post('https://translate.api.cloud.yandex.net/translate/v2/translate', headers=HEADERS, data=str(data))
    json_text = json.loads(response.text)

    return json_text["translations"][0]["text"]

In [4]:
test = translate_with_yandex("to go away")

In [5]:
test

'чтобы уйти'